In [8]:
import re
import os
import json

In [9]:
folder_path = 'data_cleaned'
cars = []


for filename in os.listdir(folder_path):
    #print("Procesando archivo: ", filename)
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r', encoding='utf-8') as f:
            try:
                data = json.load(f)
                #print(f'Hay {len(data)} coches')
                cars.extend(data)
            except json.JSONDecodeError as e:
                print(f"Error al procesar {filename}: {e}")

print(f'ARCHIVOS PROCESADOS: {len(os.listdir(folder_path))} TOTAL: {len(cars)}')

ARCHIVOS PROCESADOS: 20 TOTAL: 17741


In [10]:
def process_technical_details(car):
    for str in car["Ficha Técnica"]:
        if re.match(r'^Acabado', str):
            car["jato_classification"] = re.search(r'clasificación JATO: ([\w\s]+)', car["Ficha Técnica"][0])
    return car

'''
bodyType 1 = Berlina
bodyType 2 = Coupe
bodyType 3 = Cabrio
bodyType 4 = Familiar
bodyType 5 = Monovolumen
bodyType 6 = SUV
bodyType 7 = Pick Up
bodyType 8 = 
'''
my_set = []
for car in cars:
    if "Ficha Técnica" in car: process_technical_details(car)
    my_set.append(car["bodyTypeId"])
print(set(my_set))


{None, 1, 2, 3, 4, 5, 6, 7, 8}
